In [14]:
import pandas as pd
import re
import os

# abrev. to full name
fullname = {"Ala.":"Alabama", "Ariz.":"Arizona", "Ark.":"Arkansas",
            "Calif.":"California", "Colo.":"Colorado", "Conn.":"Connecticut",
            "DC":"D.C.", "DE":"Delaware", "Del.":"Delaware",
            "Fla.":"Florida", 
            "GA":"Georgia", "Ga.":"Georgia", 
            "IN":"Indiana", "Ind.":"Indiana", "Ill":"Illinois", "Ill.":"Illinois", "ILL":"Illinois", "ILL.":"Illinois",
            "Kan.":"Kansas", "Ky.":"Kentucky",
            "La.":"Louisiana",
            "Mass.":"Massachusetts", "Md.":"Maryland", "MD":"Maryland", "Mich.":"Michigan", "Minn.":"Minnesota", 
            "MN":"Minnesota", "Miss.":"Mississippi", "Mo.":"Missouri",
            "N.C.":"North Carolina", "NC":"North Carolina", "N.D.":"North Dakota", "Neb.":"Nebraska", "N.H":"New Hampshire",
            "N.J.":"New Jersey", "N.Y.":"New York", "NY":"New York",
            "OH": "Ohio", 
            "S.C.":"South Carolina", "SC":"South Carolina", "S.C":"South Carolina", "S.D.":"South Dakota",
            "Tenn.":"Tennessee", 
            "Pa.":"Pennsylvania", "PA":"Pennsylvania",
            "Wash.":"Washington", "Washington D.C.":"D.C.", "W.Va.":"West Virginia", 
            "Va.":"Virginia", 
            "Wis.":"Wisconsin", "WI":"Wisconsin"}


# turn ft to inch
def to_inches(height):
    if height is None:
        return None
        
    if isinstance(height, str) or not pd.isna(height):
        match = re.match(r"(\d+)' (\d+)''", height)
        if match:
            ft = int(match.group(1))
            inch = int(match.group(2))
            return ft * 12 + inch
    else:
        return None

# function to extract state  
def extract_state(hometown):
        state = ""
        if isinstance(hometown, str):
            if "/" in hometown:
                home = hometown.split("/")[0]
                if "," in home:
                    state = home.split(",")[1]
                else:
                    state = home
            else:
                if "," in hometown:
                    state = hometown.split(",")[1]

            state = state.strip()  
            if state in fullname.keys():
                state = fullname[state]
            return state
        else:
            return None

# Use abreviation to represent position
def abrev_position(pos):
    if len(pos) != 3:
        pos = pos[0]
    return pos
        
# cleaning
def clean(s, df):
    # Position
    df["Position"] = df["Position"].apply(abrev_position)
    
    # Height
    df["Height"] = df["Height"].apply(to_inches)

    # Hometown
    df["Hometown"] = df["Hometown"].apply(extract_state)

    # write to csv
    df.to_csv(f'./clean/{s}_clean.csv', index=False)
    print(f"{s} finished!")

In [26]:
# extract weight from weight string (delete unit)
def extract_weight(weight_str):
    if isinstance(weight_str, str):
        weight = int(re.findall(r'\d+', weight_str)[0])
    else:
        weight = weight_str
    return weight
    


# turn height to correct format
def correct_format(height):
    height = str(height) 
    if isinstance(height, str) or not pd.isnan(height):
        match = re.match(r"(\d+)'(\d+)\"", height)
        if match:
            ft = int(match.group(1))
            inch = int(match.group(2))
            return f"{match.group(1)}' {match.group(2)}''"
    else:
        return None

# pre-process for second group
def pre_process(df, s):
    # drop unnamed cols and redundant rows, and add index col
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(subset=['Name'])    
    df = df.reset_index(drop=False)
    df.rename(columns={"index": "Index"}, inplace=True)

    # add School
    df["School"] = s

    # add Previous School
    if "Previous School" not in df.columns:
        df["Previous School"] = None
        
    # drop useless columns
    required = ['Index', 'Name', 'Position', 'Height', 'Weight', 'Academic Year', 'Hometown', 'Previous School', 'Year', 'School']
    df = df[required]
    
    # Weight/height
    df["Weight"] = df["Weight"].apply(extract_weight)
    df["Height"] = df["Height"].apply(correct_format)
    
    return df

In [119]:
# first group of schools
schools1 = ["Indiana", "Michigan", "Minnesota", "Northwestern", "Ohio_State", "Purdue"]
for s in schools1:
    # read files
    df = pd.read_csv(f"./raw_data/{s}_basketball_players.csv")
    df.rename(columns={"Hometown / High School": "Hometown"}, inplace=True)
    clean(s, df)

Indiana finished!
Michigan finished!
Minnesota finished!
Northwestern finished!
Ohio_State finished!
Purdue finished!


In [120]:
# second group of schools
schools2 = ["Illinois", "Iowa", "Maryland", "Michigan_State", "Nebraska", "Penn_State", "Rutgers", "Wisconsin"]
for s in schools2:
    # read files
    df = pd.read_csv(f"./raw_data/{s}_basketball_players.csv")
    df = pre_process(df, s)
    clean(s, df)

Illinois finished!
Iowa finished!
Maryland finished!
Michigan_State finished!
Nebraska finished!
Penn_State finished!
Rutgers finished!
Wisconsin finished!


In [121]:
# combine files
folder_path = './clean'

# List all files in the folder that contain "clean" in the filename
all_files = [file for file in os.listdir(folder_path) if 'clean' in file]

# Read each file and store them in a list
dataframes = [pd.read_csv(os.path.join(folder_path, file)) for file in all_files]

# concatenate all dataframes into a single dataframe
all_data = pd.concat(dataframes, ignore_index=True)
all_data.drop("Index", axis=1, inplace=True)

# write to file
all_data.to_csv(f'./clean/all_players.csv', index=False)
print("All data combine successfullly!")

All data combine successfullly!


In [6]:
# combine files
folder_path = './ws'

# List all files in the folder that contain "clean" in the filename
all_files = [file for file in os.listdir(folder_path) if 'all' not in file]

# Read each file and store them in a list
dataframes = [pd.read_csv(os.path.join(folder_path, file)) for file in all_files]

# concatenate all dataframes into a single dataframe
all_data = pd.concat(dataframes, ignore_index=True)
all_data["MPG"] = all_data["MP"] / all_data["G"]

# write to file
all_data.to_csv(f'./ws/all_winshare.csv', index=False)
print("All win share data combine successfullly!")

All win share data combine successfullly!


In [9]:
# combine files
folder_path = './nba_data'

# List all files in the folder that contain "clean" in the filename
all_files = [file for file in os.listdir(folder_path) if 'all' not in file]

# Read each file and store them in a list
dataframes = [pd.read_csv(os.path.join(folder_path, file)) for file in all_files]

all_data = pd.concat(dataframes, ignore_index=True)

# write to file
all_data.to_csv(f'./nba_data/nba_bio.csv', index=False)
print("All nba bio data combine successfullly!")

All nba bio data combine successfullly!


In [35]:
def convert_position_format(position):
    if isinstance(position, str):
        return position.replace('-', '/')
    return position 

df = pd.read_csv(f"./nba_data/nba_bio.csv")
# df["Height"] = df["Height"].apply(to_inches2)
# df["Weight"] = df["Weight"].apply(extract_weight)
df["Position"] = df["Position"].apply(convert_position_format)
df.to_csv(f"./nba_data/nba_bio.csv", index=False)

,Position,Height,Weight
0,F,80.0,243
1,C,83.0,265
2,C/F,81.0,255
3,G,77.0,215
4,F/C,84.0,215
...,...,...,...
519,G,77.0,205
520,F,80.0,220
521,F,76.0,230
522,F/G,80.0,210
